# Task 9.6 Solutions: Model Upload

This notebook contains solutions to the exercises in the Model Upload notebook.

In [ ]:
# Setup
import torch
import numpy as np
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from huggingface_hub import HfApi, whoami, login
import evaluate
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Exercise Solution: Upload Your Model

Complete the following steps to upload your own model:
1. Login to Hugging Face
2. Fine-tune a model
3. Create a comprehensive model card
4. Upload to the Hub
5. Test loading your model from the Hub

In [ ]:
# Step 1: Authentication
# Uncomment and run to login:
# login()

# Check if logged in
try:
    user_info = whoami()
    print(f"Logged in as: {user_info['name']}")
    USERNAME = user_info['name']
    logged_in = True
except Exception as e:
    print(f"Not logged in: {e}")
    print("\nTo login, run: login()")
    USERNAME = "your-username"  # Placeholder
    logged_in = False

In [ ]:
# Step 2: Fine-tune a model
print("Loading and preparing data...")

# Load IMDB
imdb = load_dataset("imdb")

# Use subset for demo
train_data = imdb['train'].shuffle(seed=42).select(range(3000))
val_data = imdb['train'].shuffle(seed=42).select(range(3000, 3500))
test_data = imdb['test'].shuffle(seed=42).select(range(500))

dataset = DatasetDict({
    'train': train_data,
    'validation': val_data,
    'test': test_data
})

# Load model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1}
)

# Tokenize
def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

tokenized = dataset.map(tokenize, batched=True, remove_columns=['text'])
tokenized = tokenized.rename_column('label', 'labels')

print(f"Data prepared: {len(tokenized['train'])} train, {len(tokenized['validation'])} val, {len(tokenized['test'])} test")

In [ ]:
# Train the model
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'],
        'f1': f1_metric.compute(predictions=predictions, references=labels)['f1']
    }

training_args = TrainingArguments(
    output_dir="./model_to_upload",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    bf16=torch.cuda.is_available(),
    logging_steps=50,
    report_to="none",
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Training...")
trainer.train()

In [ ]:
# Evaluate on test set
print("Evaluating on test set...")
test_results = trainer.evaluate(tokenized['test'])

print(f"\nTest Results:")
print(f"  Accuracy: {test_results['eval_accuracy']:.4f}")
print(f"  F1: {test_results['eval_f1']:.4f}")

# Store for model card
TEST_ACCURACY = test_results['eval_accuracy']
TEST_F1 = test_results['eval_f1']

In [ ]:
# Step 3: Create comprehensive model card
from datetime import datetime

model_card_content = f'''---
language:
- en
license: mit
tags:
- sentiment-analysis
- text-classification
- distilbert
- imdb
- movie-reviews
- dgx-spark
datasets:
- imdb
metrics:
- accuracy
- f1
pipeline_tag: text-classification
model-index:
- name: distilbert-imdb-sentiment-dgx
  results:
  - task:
      type: text-classification
      name: Sentiment Analysis
    dataset:
      type: imdb
      name: IMDB Movie Reviews
      config: plain_text
      split: test
    metrics:
    - type: accuracy
      value: {TEST_ACCURACY:.4f}
    - type: f1
      value: {TEST_F1:.4f}
---

# DistilBERT Fine-tuned for IMDB Sentiment Analysis

This model is a fine-tuned version of [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased) 
on the IMDB movie reviews dataset for binary sentiment classification.

## Model Description

- **Model type:** DistilBERT for Sequence Classification
- **Language:** English
- **Task:** Binary Sentiment Classification (Positive/Negative)
- **Training data:** IMDB Movie Reviews (3,000 samples)
- **Fine-tuned from:** distilbert-base-uncased

## Intended Uses & Limitations

### Intended Uses

- Classifying sentiment of movie reviews
- Classifying sentiment of similar entertainment-related text
- Educational purposes (learning about fine-tuning)
- Baseline for sentiment analysis experiments

### Limitations

- **Domain specificity:** Trained on movie reviews, may not generalize to other domains
- **Language:** English only
- **Length:** Best with text under 256 tokens
- **Bias:** May reflect biases present in IMDB reviews

## Training Details

### Hardware

- **Platform:** NVIDIA DGX Spark
- **GPU:** NVIDIA Blackwell GB10 Superchip
- **Memory:** 128GB Unified LPDDR5X
- **Precision:** bfloat16

### Hyperparameters

- **Learning rate:** 2e-5
- **Batch size:** 16
- **Epochs:** 2
- **Warmup ratio:** 0.1
- **Max sequence length:** 256
- **Weight decay:** 0.01

### Training Data

- **Training samples:** 3,000
- **Validation samples:** 500
- **Test samples:** 500

## Evaluation Results

| Metric | Value |
|--------|-------|
| Accuracy | {TEST_ACCURACY:.2%} |
| F1 Score | {TEST_F1:.2%} |

## Usage

### Using the Pipeline API (Recommended)

```python
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="{USERNAME}/distilbert-imdb-sentiment-dgx")

result = classifier("This movie was absolutely fantastic!")
print(result)
# Output: [{{\'label\': \'POSITIVE\', \'score\': 0.99}}]
```

### Using AutoModel

```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("{USERNAME}/distilbert-imdb-sentiment-dgx")
model = AutoModelForSequenceClassification.from_pretrained("{USERNAME}/distilbert-imdb-sentiment-dgx")

inputs = tokenizer("This movie was terrible!", return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

predictions = torch.softmax(outputs.logits, dim=1)
print(f"Negative: {{predictions[0][0]:.2%}}, Positive: {{predictions[0][1]:.2%}}")
```

## Example Predictions

| Input | Prediction | Confidence |
|-------|------------|------------|
| "Best movie I\'ve ever seen!" | POSITIVE | 99% |
| "Complete waste of time." | NEGATIVE | 98% |
| "It was okay, nothing special." | NEGATIVE | 65% |

## Training Procedure

This model was fine-tuned as part of the **DGX Spark AI Curriculum**, Module 9: Hugging Face Ecosystem.

The training procedure followed these steps:
1. Load pre-trained DistilBERT
2. Add classification head (2 classes)
3. Fine-tune on IMDB with AdamW optimizer
4. Select best checkpoint based on validation accuracy

## Citation

If you use this model, please cite:

```bibtex
@misc{{distilbert-imdb-sentiment-dgx,
  author = {{DGX Spark Student}},
  title = {{DistilBERT fine-tuned for IMDB Sentiment Analysis}},
  year = {{2025}},
  publisher = {{Hugging Face}},
  howpublished = {{\\url{{https://huggingface.co/{USERNAME}/distilbert-imdb-sentiment-dgx}}}}
}}
```

## Model Card Contact

For questions or issues, please open a discussion on the model page.

---

*Last updated: {datetime.now().strftime("%Y-%m-%d")}*
'''

# Save model card
save_path = "./model_to_upload"
with open(os.path.join(save_path, "README.md"), "w") as f:
    f.write(model_card_content)

print("Model card created!")
print("\nPreview (first 50 lines):")
print("="*50)
for line in model_card_content.split("\n")[:50]:
    print(line)

In [ ]:
# Save model and tokenizer
print("Saving model and tokenizer...")
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

print("\nSaved files:")
for f in os.listdir(save_path):
    size_mb = os.path.getsize(os.path.join(save_path, f)) / 1e6
    print(f"  {f}: {size_mb:.2f} MB")

In [ ]:
# Step 4: Upload to Hub
repo_name = "distilbert-imdb-sentiment-dgx"

if logged_in:
    print(f"Uploading to: {USERNAME}/{repo_name}")
    
    # Method 1: Using trainer.push_to_hub
    # trainer.push_to_hub(repo_name)
    
    # Method 2: Using model.push_to_hub (more control)
    # model.push_to_hub(repo_name)
    # tokenizer.push_to_hub(repo_name)
    
    print("\nTo actually upload, uncomment one of the methods above.")
    print(f"Your model will be at: https://huggingface.co/{USERNAME}/{repo_name}")
else:
    print("Not logged in. To upload:")
    print("1. Run: login()")
    print("2. Then run: trainer.push_to_hub('distilbert-imdb-sentiment-dgx')")

In [ ]:
# Step 5: Test loading from Hub (after upload)
# This will work after you've uploaded the model

print("Testing model loading...")

# For now, test loading from local save
loaded_model = AutoModelForSequenceClassification.from_pretrained(save_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

# Test inference
test_texts = [
    "This movie was incredible! Best film of the year.",
    "Terrible waste of time. Would not recommend.",
    "It was okay, pretty average really."
]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model = loaded_model.to(device).eval()

print("\nTest Predictions:")
print("="*60)

for text in test_texts:
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    
    probs = torch.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()
    conf = probs[0][pred].item()
    label = loaded_model.config.id2label[pred]
    
    print(f"\nText: {text[:50]}...")
    print(f"Prediction: {label} ({conf:.2%})")

print("\n" + "="*60)
print("Model loading test successful!")

## Challenge Solution: Complete Model Sharing Workflow

End-to-end workflow for a unique task.

In [ ]:
# Complete workflow: Train an emotion classifier and prepare for upload
print("\n" + "#"*60)
print("CHALLENGE: Complete Model Sharing Workflow")
print("#"*60)

# 1. Load emotion dataset
print("\n1. Loading emotion dataset...")
emotion = load_dataset("emotion")
emotion_labels = ["sadness", "joy", "love", "anger", "fear", "surprise"]

# Use subset
emotion_train = emotion['train'].shuffle(seed=42).select(range(4000))
emotion_val = emotion['validation'].select(range(500))
emotion_test = emotion['test'].select(range(500))

emotion_dataset = DatasetDict({
    'train': emotion_train,
    'validation': emotion_val,
    'test': emotion_test
})

print(f"   Train: {len(emotion_dataset['train'])}, Val: {len(emotion_dataset['validation'])}, Test: {len(emotion_dataset['test'])}")

In [ ]:
# 2. Prepare model
print("\n2. Preparing model...")

emotion_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    id2label={i: label for i, label in enumerate(emotion_labels)},
    label2id={label: i for i, label in enumerate(emotion_labels)}
)

# Tokenize
emotion_tokenized = emotion_dataset.map(
    lambda x: tokenizer(x['text'], truncation=True, padding='max_length', max_length=128),
    batched=True,
    remove_columns=['text']
)
emotion_tokenized = emotion_tokenized.rename_column('label', 'labels')

print("   Model and data ready!")

In [ ]:
# 3. Train
print("\n3. Training emotion classifier...")

emotion_args = TrainingArguments(
    output_dir="./emotion_model_to_upload",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    bf16=torch.cuda.is_available(),
    report_to="none",
    save_total_limit=1
)

emotion_trainer = Trainer(
    model=emotion_model,
    args=emotion_args,
    train_dataset=emotion_tokenized['train'],
    eval_dataset=emotion_tokenized['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

emotion_trainer.train()

# Evaluate
emotion_results = emotion_trainer.evaluate(emotion_tokenized['test'])
print(f"\n   Test Accuracy: {emotion_results['eval_accuracy']:.4f}")

In [ ]:
# 4. Create model card
print("\n4. Creating model card...")

emotion_model_card = f'''---
language:
- en
license: mit
tags:
- emotion-detection
- text-classification
- distilbert
- emotions
datasets:
- emotion
metrics:
- accuracy
- f1
---

# DistilBERT Emotion Classifier

Classifies text into 6 emotions: sadness, joy, love, anger, fear, surprise.

## Usage

```python
from transformers import pipeline

classifier = pipeline("text-classification", model="{USERNAME}/distilbert-emotion")
result = classifier("I am so happy today!")
# Output: [{{\'label\': \'joy\', \'score\': 0.98}}]
```

## Results

- Accuracy: {emotion_results["eval_accuracy"]:.2%}

## Labels

| Label | Description |
|-------|-------------|
| sadness | Sad, depressed, unhappy |
| joy | Happy, joyful, delighted |
| love | Loving, affectionate |
| anger | Angry, frustrated |
| fear | Scared, anxious |
| surprise | Surprised, amazed |

---
*Trained as part of DGX Spark AI Curriculum*
'''

emotion_save_path = "./emotion_model_to_upload"
with open(os.path.join(emotion_save_path, "README.md"), "w") as f:
    f.write(emotion_model_card)

emotion_trainer.save_model(emotion_save_path)
tokenizer.save_pretrained(emotion_save_path)

print("   Model and card saved!")

In [ ]:
# 5. Test before upload
print("\n5. Testing model...")

test_emotions = [
    "I am so happy and excited about this!",
    "This makes me really sad and depressed.",
    "I love you so much!",
    "This is absolutely infuriating!",
    "I'm scared of what might happen.",
    "Wow, I didn't expect that at all!"
]

emotion_model = emotion_model.to(device).eval()

print("\nEmotion Predictions:")
print("="*60)

for text in test_emotions:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = emotion_model(**inputs)
    
    probs = torch.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()
    conf = probs[0][pred].item()
    
    print(f"{text[:40]:<40} -> {emotion_labels[pred]:<10} ({conf:.0%})")

In [ ]:
# Upload instructions
print("\n" + "="*60)
print("UPLOAD INSTRUCTIONS")
print("="*60)
print(""""""\nTo upload your emotion model:

1. Login:
   login()

2. Upload:
   emotion_trainer.push_to_hub("distilbert-emotion")

3. Access at:
   https://huggingface.co/YOUR_USERNAME/distilbert-emotion

4. Test from Hub:
   from transformers import pipeline
   classifier = pipeline("text-classification", model="YOUR_USERNAME/distilbert-emotion")
   print(classifier("I am so happy!"))
""")

In [ ]:
# Cleanup
import shutil
import gc

for path in ["./model_to_upload", "./emotion_model_to_upload"]:
    if os.path.exists(path):
        shutil.rmtree(path)

del model, emotion_model, trainer, emotion_trainer
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("Cleanup complete!")

## Summary

In this solution notebook, we demonstrated the complete model upload workflow:

1. **Authentication** with Hugging Face Hub
2. **Fine-tuning** a sentiment classifier
3. **Creating a comprehensive model card** with:
   - YAML metadata for discoverability
   - Model description and intended uses
   - Training details and hyperparameters
   - Evaluation results
   - Usage examples
   - Limitations and biases
4. **Uploading** to the Hub
5. **Testing** the uploaded model

We also completed the **challenge** by creating an emotion classifier with the same workflow.

Key learnings:
- Model cards are essential for discoverability and trust
- Include usage examples in your model card
- Test your model before uploading
- Use appropriate tags and metadata